# <span style="color:red">**exercise 4**</span>
---

### submited by:
- Name: Shahar Asher
- Id: 209305408
- Email adress: shaharas@edu.hac.ac.il
- Date: 12/05/2024

### Operation system: Windows 11
### Python version: 3.11.5 (Using anaconda)
### IDE: Visual Studio Code
### libraries: typing, numpy, pillow (PIL)
---

In [12]:
# imports

from typing import Final
import re
import time
import inspect
from functools import wraps
from contextlib import ContextDecorator

# Q.1


In [13]:
# global variables
PROFILE:bool = True
HEADER_PRINTER:bool = False

In [14]:
def profile_decorator(func: callable) -> callable:
    # Initialize a counter for function calls
    func._call_count = 0

    @wraps(func)
    def wrapper(*args, **kwargs) -> callable:
        global HEADER_PRINTER
        
        if not PROFILE:
            return func(*args, **kwargs)

        # Increment the function call counter
        func._call_count += 1

        # Get the frame information of the caller
        frame: Final[object] = inspect.currentframe().f_back
        filename: Final[str] = frame.f_code.co_filename
        filename = re.search(r"(\w+.py)", filename)
        filename = filename.group()
        lineno: Final[int] = frame.f_lineno

        # Start measuring CPU and wall time
        start_cpu: Final[float] = time.process_time_ns()
        start_wall: Final[float] = time.perf_counter_ns()

        # Execute the function
        result = func(*args, **kwargs)

        # Stop measuring CPU and wall time
        end_cpu: Final[float] = time.process_time_ns()
        end_wall: Final[float] = time.perf_counter_ns()

        # Calculate the elapsed times
        cpu_time: Final[float] = (end_cpu - start_cpu) / 1e9  # Convert nanoseconds to seconds
        wall_time: Final[float] = (end_wall - start_wall) / 1e9  # Convert nanoseconds to seconds

        # Log the details in table format
        if not HEADER_PRINTER:
            print(f"{'Function name':<15} {'filename':<15} {'line':<5} {'count':<5} {'cpu':<10} {'wall':<10}")
            HEADER_PRINTER = True

        print(f"{func.__name__:<15} {filename:<15} {lineno:<5} {func._call_count:<5} {cpu_time:<10.6f} {wall_time:<10.5f}")

        return result

    return wrapper

In [15]:
@profile_decorator
def func1():
    total:int = 0
    for _ in range(1000000):
        total += sum(i * i for i in range(100))


@profile_decorator
def func2():
    total:int = 0
    for _ in range(1000000):
        total += sum(i * i for i in range(100))

In [16]:
# call to func1
func1()
func1()
func1()

# call to func2
func2()
func2()

Function name   filename        line  count cpu        wall      
func1           2948867115.py   2     1     3.875000   4.71608   
func1           2948867115.py   3     2     3.843750   4.80118   
func1           2948867115.py   4     3     4.468750   5.16818   
func2           2948867115.py   7     1     4.375000   4.86140   
func2           2948867115.py   8     2     4.515625   4.75220   


---
# Q.2


In [17]:
# global variables
PROFILE:bool = True
HEADER_PRINTER:bool = False

In [18]:
class ProfileBlock(ContextDecorator):
    def __init__(self, func_name="ProfileBlock"):
        self.func_name = func_name

    def __enter__(self):
        global HEADER_PRINTER

        if not PROFILE:
            return self

        # Get the frame information of the caller
        self.frame = inspect.currentframe().f_back
        self.filename = self.frame.f_code.co_filename
        self.filename = re.search(r"(\w+.py)", self.filename)
        self.filename = self.filename.group()
        self.lineno = self.frame.f_lineno

        # Start measuring CPU and wall time
        self.start_cpu = time.process_time()
        self.start_wall = time.perf_counter()

        # Initialize call count
        if not hasattr(self, 'call_count'):
            self.call_count = 0

        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if not PROFILE:
            return

        # Stop measuring CPU and wall time
        end_cpu = time.process_time()
        end_wall = time.perf_counter()

        # Calculate the elapsed times
        cpu_time = end_cpu - self.start_cpu
        wall_time = end_wall - self.start_wall

        # Increment the call count
        self.call_count += 1

        # Print the header if not already printed
        global HEADER_PRINTER
        if not HEADER_PRINTER:
            print(f"{'Function name':<15} {'filename':<15} {'line':<5} {'count':<5} {'cpu':<10} {'wall':<10}")
            HEADER_PRINTER = True

        # Log the details in table format
        print(f"{self.func_name:<15} {self.filename:<15} {self.lineno:<5} {self.call_count:<5} {cpu_time:<10.6f} {wall_time:<10.5f}")

---


In [19]:
def func1():
    with ProfileBlock("func1"):
        total = 0
        for _ in range(1000000):
            total += sum(i * i for i in range(100))


def func2():
    with ProfileBlock("func2"):
        total = 0
        for _ in range(1000000):
            total += sum(i * i for i in range(100))

In [20]:
with ProfileBlock("main block"):
    # call to func1
    func1()
    func1()
    func1()

    # call to func2
    func2()
    func2()

Function name   filename        line  count cpu        wall      
func1           2138028358.py   2     1     4.328125   4.75827   
func1           2138028358.py   2     1     4.343750   4.73912   
func1           2138028358.py   2     1     3.937500   4.92149   
func2           2138028358.py   9     1     4.453125   4.96955   
func2           2138028358.py   9     1     4.156250   5.28505   
main block      1638514948.py   1     1     21.218750  24.67421  
